# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
#Create a Cluster
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandradm 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassandradm')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [29]:
## Create Table music_hist_by_sess_id to select the artist, song title and song's length in the music app history
query = "CREATE TABLE IF NOT EXISTS music_hist_by_sess_id "
query = query + "(sessionId int, itemInSession int, artist text, song text,length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [30]:
# Insert data to music_hist_by_sess_id table 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO music_hist_by_sess_id(sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [31]:
## Select statement to verify the data was entered into the table 
## for sessionId = 338, and itemInSession = 4
query = "select artist, song, length from music_hist_by_sess_id WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [32]:
## Create Table music_hist_by_user_id to select name of artist, song (sorted by itemInSession) and user
query1 = "CREATE TABLE IF NOT EXISTS music_hist_by_user_id "
query1 = query1 + "(userId int, sessionId int, itemInSession int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)                   

In [33]:
## Insert Data to music_hist_by_user_id table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO music_hist_by_user_id(userId, sessionId, itemInSession, artist, song, first_name, last_name)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [34]:
## Select statement to verify the data was entered into the table
## for userid = 10, sessionid = 182
query1 = "select artist, song, first_name, last_name from music_hist_by_user_id WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [35]:
## Create table music_hist_by_song to select every user name in my music app history 
query2 = "CREATE TABLE IF NOT EXISTS music_hist_by_song "
query2 = query2 + "(song text, userId int, first_name text, last_name text, PRIMARY KEY (song, userId))"
try:
    session.execute(query2)
except Exception as e:
    print(e)                     

In [36]:
## Insert Data to music_hist_by_song table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO music_hist_by_song(song, userId, first_name, last_name)"
        query2 = query2 + " VALUES (%s, %s, %s, %s)"
        session.execute(query2, (line[9], int(line[10]), line[1], line[4]))

In [37]:
## Select statement to verify the data was entered into the table
## who listened to the song 'All Hands Against His Own'
query2 = "select userId, first_name, last_name from music_hist_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.first_name, row.last_name)

29 Jacqueline Lynch
80 Tegan Levine
95 Sara Johnson


### Drop the tables before closing out the sessions

In [38]:
## Drop the tables music_hist_by_song, music_hist_by_user_id, music_hist_by_sess_id 
query = "drop table music_hist_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table music_hist_by_user_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table music_hist_by_sess_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [39]:
session.shutdown()
cluster.shutdown()